In [1]:
data_var = '2023-12-13'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7989,2023-12-13,Brasil Nbb,20:00,Pinheiros,R10 Score Vasco da Gama,2.20,1.62,148.5,1.85,1.85,1.5,2.00,1.72,fTEHSSHp,0.454545,0.617284,0.540541,0.540541,0.071829,0.0,0.0,NaN,NaN,174.420,57.099556,0.327368,175.192,44.923113,0.256422,0.000,0.000,142.35,203.68,0.0,0.0,0.0,0.0,0.214724,0.000000,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7990,2023-12-13,Espanha Acb,16:30,Barcelona,Manresa,1.11,6.95,167.5,1.81,1.97,-14.5,2.06,1.73,OOES7bTf,0.900901,0.143885,0.552486,0.507614,0.044786,0.0,0.4,NaN,NaN,111.046,15.536154,0.139907,252.020,149.420669,0.592892,0.000,313.960,85.60,477.40,0.0,0.0,0.0,0.0,1.024691,0.059861,0.123137,0.00,0.000,inf,0.000000,0.0,0.000000,-2.73,-0.546,-10.897436,0.000000,0.0,0.000000
7991,2023-12-13,Eua Nba,00:00,Phoenix Suns,Golden State Warriors,1.93,2.01,227.5,1.82,1.92,-1.5,2.09,1.87,j93HZUfR,0.518135,0.497512,0.549451,0.520833,0.015647,0.8,0.6,NaN,NaN,202.002,56.037958,0.277413,255.490,80.709877,0.315902,183.406,289.788,224.72,345.44,0.0,0.0,0.0,0.0,0.028715,0.037813,0.078567,0.61,0.122,7.622951,0.636100,0.5,-0.136100,-5.00,-1.000,-1.010000,0.482016,0.4,-0.082016
7992,2023-12-13,Eua Nba,00:30,Los Angeles Clippers,Sacramento Kings,1.67,2.41,234.5,1.89,2.04,-3.5,2.01,1.93,8ljqVl3r,0.598802,0.414938,0.529101,0.490196,0.013740,0.0,0.2,NaN,NaN,193.228,55.906484,0.289329,235.172,44.380450,0.188715,162.066,228.496,171.76,209.76,0.0,0.0,0.0,0.0,0.256500,0.053978,0.028715,-0.21,-0.042,-15.952381,0.679088,0.6,-0.079088,1.27,0.254,5.551181,0.513331,0.7,0.186669
7993,2023-12-13,Eua Nba,21:00,Detroit Pistons,Philadelphia 76ers,5.95,1.18,231.5,1.82,2.06,10.5,2.10,1.84,nukmU8Il,0.168067,0.847458,0.549451,0.485437,0.015525,0.8,0.8,NaN,NaN,473.776,241.447603,0.509624,220.870,94.288825,0.426897,315.884,254.340,397.29,162.44,1.0,0.0,0.0,0.0,0.946115,0.087477,0.093324,-5.00,-1.000,-4.950000,0.371779,0.1,-0.271779,-1.76,-0.352,-0.511364,0.520299,0.5,-0.020299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8093,2023-12-13,Eua Ncaa,22:00,Northwestern,Chicago St,1.02,19.00,133.5,1.91,1.91,-26.5,2.05,1.71,Y1deSFxH,0.980392,0.052632,0.523560,0.523560,0.033024,0.8,0.4,NaN,NaN,144.258,94.317276,0.653810,293.052,227.437415,0.776099,139.284,356.124,92.82,689.00,0.0,1.0,0.0,0.0,1.270108,0.000000,0.127881,3.22,0.644,0.031056,0.000000,0.0,0.000000,-2.37,-0.474,-37.974684,0.000000,0.0,0.000000
8094,2023-12-13,Eua Ncaa,22:30,La Lafayette,Eastern Kentucky,1.76,2.14,150.5,1.91,1.91,-3.5,2.05,1.71,4xaZvLJK,0.568182,0.467290,0.523560,0.523560,0.035472,0.0,0.0,NaN,NaN,177.920,68.354177,0.384185,NaN,NaN,NaN,0.000,0.000,241.96,316.20,0.0,0.0,0.0,0.0,0.137795,0.000000,0.127881,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8095,2023-12-13,Eua Ncaa,23:00,UNLV,Creighton,7.70,1.10,150.5,1.91,1.91,11.5,2.10,1.67,x8A5UcUu,0.129870,0.909091,0.523560,0.523560,0.038961,0.8,0.0,NaN,NaN,135.134,19.636994,0.145315,108.574,35.858883,0.330271,122.994,0.000,125.25,150.41,0.0,0.0,0.0,0.0,1.060660,0.000000,0.161303,-2.23,-0.446,-15.022422,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8096,2023-12-13,Europa Liga Waba Feminina,14:00,Plamen Pozega F,Orlovi F,8.80,1.05,159.5,1.81,1.89,16.5,1.93,1.74,zw1lMGVS,0.113636,0.952381,0.552486,0.529101,0.066017,0.0,0.

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
2,00:00,Eua Nba,Phoenix Suns,Golden State Warriors,227.5,1.82,0.9397,Over
3,00:30,Eua Nba,Los Angeles Clippers,Sacramento Kings,234.5,1.89,1.0000,Over
4,21:00,Eua Nba,Detroit Pistons,Philadelphia 76ers,231.5,1.82,0.9999,Over
5,21:00,Eua Nba,Washington Wizards,New Orleans Pelicans,239.5,1.86,1.0000,Over
6,21:30,Eua Nba,Miami Heat,Charlotte Hornets,222.5,1.81,0.9999,Over
7,21:30,Eua Nba,Toronto Raptors,Atlanta Hawks,237.5,1.81,1.0000,Over
8,22:00,Eua Nba,Houston Rockets,Memphis Grizzlies,211.5,1.86,1.0000,Over
9,22:00,Eua Nba,Milwaukee Bucks,Indiana Pacers,257.5,1.83,1.0000,Over
11,23:00,Eua Nba,Phoenix Suns,Brooklyn Nets,228.5,1.97,0.9998,Over
12,23:00,Eua Nba,Utah Jazz,New York Knicks,225.5,1.84,1.0000,Over
